In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
# Set the plotting style
sns.set_style("whitegrid")

# Define the absolute path to your file.
# NOTE: This path is hardcoded based on your input. Please ensure this path 
# is correct on your local machine before running the script.
DATA_FILE_PATH = r"C:\Users\9a33n\OneDrive\Documents\Sample - Superstore.csv" 
ENCODING_TYPE = 'unicode_escape' # Best for Superstore dataset

# --- 1. DATA LOADING AND CLEANING ---

try:
    # Load the data using the absolute file path
    df = pd.read_csv(DATA_FILE_PATH, encoding=ENCODING_TYPE)
    print(f"Data loaded successfully from {DATA_FILE_PATH}. Total rows: {len(df)}")
except FileNotFoundError:
    print(f"Error: {DATA_FILE_PATH} not found. Please ensure the path is correct and the file exists.")
    exit()

# Convert Date columns to datetime objects for calculations
# Note: Using format='%m/%d/%Y' based on common Superstore date format (e.g., 11/8/2016)
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%m/%d/%Y')

# Feature Engineering: Calculate Inventory Proxy (Days to Ship)
# This is used as a proxy for inventory turnover time or logistical complexity.
df['Days_to_Ship'] = (df['Ship Date'] - df['Order Date']).dt.days

# Feature Engineering: Extract time features for Seasonality
df['Order_Month'] = df['Order Date'].dt.to_period('M')

# --- 2. CORRELATION ANALYSIS (Inventory Proxy vs. Profitability) ---
print("\n--- Starting Correlation Analysis ---")

# Aggregate key metrics by Sub-Category, as requested for correlation
sub_category_analysis = df.groupby('Sub-Category').agg(
    Avg_Ship_Days=('Days_to_Ship', 'mean'),  # X-axis variable (Inventory Proxy)
    Total_Profit=('Profit', 'sum'),
    Total_Sales=('Sales', 'sum')
).reset_index()

# Calculate Profit Margin for correlation (Y-axis variable)
sub_category_analysis['Profit_Margin'] = (
    sub_category_analysis['Total_Profit'] / sub_category_analysis['Total_Sales']
)

# Calculate and Print Pearson Correlation Coefficient
correlation = sub_category_analysis[
    ['Avg_Ship_Days', 'Profit_Margin']
].corr().iloc[0, 1]

print(f"Correlation between Avg Ship Days and Profit Margin: {correlation:.4f}")
print("Interpretation: A value near 0 means very weak linear correlation.")


# Visualize the correlation (Scatter Plot)
plt.figure(figsize=(12, 7))
sns.scatterplot(
    x='Avg_Ship_Days',
    y='Profit_Margin',
    data=sub_category_analysis,
    s=150, # Size of dots
    hue='Profit_Margin', # Color by margin for better visual insight
    palette="RdYlGn" # Red-Yellow-Green color mapping
)

# Annotate each point with the Sub-Category name for clear identification
for i, row in sub_category_analysis.iterrows():
    plt.annotate(
        row['Sub-Category'],
        (row['Avg_Ship_Days'] + 0.05, row['Profit_Margin']),
        fontsize=9
    )

plt.title('Sub-Category Profit Margin vs. Average Days to Ship')
plt.xlabel('Average Days to Ship (Inventory/Logistics Proxy)')
plt.ylabel('Profit Margin')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('correlation_inventory_profit.png')
plt.close()
print("Visualization saved as 'correlation_inventory_profit.png'.")

# --- 3. SEASONAL BEHAVIOR ANALYSIS ---
print("\n--- Starting Seasonal Analysis ---")

# Aggregate sales by month to see trends over time
seasonal_data = df.groupby('Order_Month').agg(
    Monthly_Sales=('Sales', 'sum')
).reset_index()

# Convert the Order_Month column to string for proper plotting on the x-axis
seasonal_data['Order_Month'] = seasonal_data['Order_Month'].astype(str)

# Plot the monthly sales trend (Time Series)
plt.figure(figsize=(16, 6))
sns.lineplot(
    x='Order_Month',
    y='Monthly_Sales',
    data=seasonal_data,
    marker='o',
    color='#0077B6' # Blue color for professionalism
)
plt.title('Monthly Sales Trend (Seasonal Behavior Analysis)')
plt.xlabel('Order Month (YYYY-MM)')
plt.ylabel('Total Monthly Sales ($)')
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.ticklabel_format(style='plain', axis='y') # Prevent scientific notation on y-axis
plt.tight_layout()
plt.savefig('seasonal_sales_trend.png')
plt.close()
print("Visualization saved as 'seasonal_sales_trend.png'.")

# --- 4. EXPORT AGGREGATED DATA (Optional, but useful for Tableau) ---
# Export the profitability data calculated in SSMS using Python for Tableau use
sub_category_analysis.to_csv('sub_category_performance_python.csv', index=False)
print("\nSub-category performance metrics saved to 'sub_category_performance_python.csv'.")

print("\nPython script execution complete. All analysis components are ready for Tableau and the final report.")

Data loaded successfully from C:\Users\9a33n\OneDrive\Documents\Sample - Superstore.csv. Total rows: 9994

--- Starting Correlation Analysis ---
Correlation between Avg Ship Days and Profit Margin: 0.0582
Interpretation: A value near 0 means very weak linear correlation.
Visualization saved as 'correlation_inventory_profit.png'.

--- Starting Seasonal Analysis ---
Visualization saved as 'seasonal_sales_trend.png'.

Sub-category performance metrics saved to 'sub_category_performance_python.csv'.

Python script execution complete. All analysis components are ready for Tableau and the final report.
